In [6]:
import numpy as np
import sympy as sp

In [ ]:
from sympy import symbols, Matrix, simplify_logic
from sympy.logic.boolalg import Not

# Define os símbolos para a matriz original
x1, x2, x3, x4 = symbols('x1 x2 x3 x4')

# Define as variáveis booleanas de interesse
x, y = symbols('x y')

# Define a matriz que representa a função booleana
M = Matrix([
    x2 * x3,
    x1 * x3 + x1 * x4 + x2 * x4
])

# Define o mapeamento corrigido:
# x1 = 1 - x, x2 = x, x3 = Not(y) = 1 - y, x4 = y
mapping = {x1: 1 - x, x2: x, x3: 1 - y, x4: y}

# Aplica a substituição na matriz
M_transformed = M.subs(mapping)

# Simplifica as expressões usando simplify_logic
expr1 = simplify_logic(M_transformed[0], force=True)
expr2 = simplify_logic(M_transformed[1], force=True)

print("Primeira expressão (x2*x3) transformada e simplificada:")
print(expr1)  # Espera-se: x & Not(y) ou x & (1 - y)

print("\nSegunda expressão (x1*x3 + x1*x4 + x2*x4) transformada e simplificada:")
print(expr2)  # Espera-se: Not(x) | y ou (1 - x) | y

Primeira expressão (x2*x3) transformada e simplificada:
x*(1 - y)

Segunda expressão (x1*x3 + x1*x4 + x2*x4) transformada e simplificada:
x*y + y*(1 - x) + (1 - x)*(1 - y)


In [39]:
from sympy import symbols

def criar_mapping(bits):
    """
    Cria um dicionário de mapeamento dinâmico para substituir as variáveis x1, x2, ..., x(2*bits)
    usando novas variáveis v1, v2, ..., v(bits).

    Parameters:
    bits (int): O número de bits, determinando 2*bits variáveis a serem mapeadas.

    Returns:
    dict: Dicionário contendo as substituições de x_i em termos de v_i.
    """
    # Criação das variáveis originais x1, x2, ..., x(2*bits)
    variaveis_originais = symbols(f'x1:{2*bits+1}')
    
    # Criação das novas variáveis v1, v2, ..., v(bits)
    novas_variaveis = symbols(f'v1:{bits+1}')
    
    # Construção do mapeamento dinâmico
    mapping = {}
    for i in range(bits):
        x_impar = variaveis_originais[2 * i]    # x1, x3, x5, ...
        x_par = variaveis_originais[2 * i + 1]  # x2, x4, x6, ...
        v = novas_variaveis[i]  # v1, v2, v3, ...

        # Mapeamento baseado no padrão identificado
        mapping[x_impar] = 1 - v
        mapping[x_par] = v

    return mapping

# Testando a função com 3 bits
bits = 2
mapping_dinamico = criar_mapping(bits)

# Aplica a substituição na matriz
M_transformed = M.subs(mapping)

# Simplifica as expressões usando simplify_logic
expr1 = simplify_logic(M_transformed[0], force=True)
expr2 = simplify_logic(M_transformed[1], force=True)

print("Primeira expressão:")
print(expr1)

print("\nSegunda expressão:")
print(expr2)

Primeira expressão:
x1*(1 - x2)

Segunda expressão:
x1*x2 + x2*(1 - x1) + (1 - x1)*(1 - x2)


In [35]:
type(expr2)

sympy.core.add.Add

In [2]:
def criar_vetor_de_simbolos(tamanho):
    """
    Cria um vetor (lista) de símbolos SymPy com o tamanho especificado.

    Parameters:
    tamanho (int): O número de símbolos a serem criados.

    Returns:
    list: Lista contendo os símbolos criados.
    """
    # Cria os símbolos usando sp.symbols
    return sp.symbols(f'x1:{tamanho+1}')

In [3]:
sp.Matrix(np.array_split(criar_vetor_de_simbolos(4), 2))

Matrix([
[x1, x2],
[x3, x4]])

In [4]:
# Função para calcular o produto Kronecker considerando binário exclusivo
def kronecker_binary(vectors):
    result = np.array([1])  # Inicia com o elemento neutro da multiplicação
    for v in vectors:
        result = np.kron(result, v)
    return result

In [5]:
kronecker_binary([[1,0],[0,1]])

array([0, 1, 0, 0])

In [6]:
bits = 2
variaveis = sp.Matrix(kronecker_binary(np.array_split(criar_vetor_de_simbolos(bits*2), bits)))
variaveis

Matrix([
[x1*x3],
[x1*x4],
[x2*x3],
[x2*x4]])

In [7]:
# Iterando pelo vetor com o índice e o valor
for linha, elemento in enumerate(variaveis):
    print(f"Linha {linha}: {elemento}")

Linha 0: x1*x3
Linha 1: x1*x4
Linha 2: x2*x3
Linha 3: x2*x4


In [3]:
def decimal_para_binario_vetor(numero_decimal, tamanho_bits=8):
    """
    Converte um número decimal para um vetor binário usando produtos de Kronecker.

    Args:
        numero_decimal (int): Número decimal a ser convertido.
        tamanho_bits (int, opcional): Número de bits para representar o número binário. Padrão é 8.

    Returns:
        np.ndarray: Vetor resultante da conversão.
    
    Raises:
        ValueError: Se o número decimal for negativo ou se o tamanho de bits for insuficiente.
    """
    # Mapeamento dos caracteres binários para os vetores desejados
    mapeamento = {'0': [1, 0], '1': [0, 1]}

    # Verifica se o número é um inteiro não negativo
    if not isinstance(numero_decimal, int):
        raise TypeError("O número decimal deve ser um inteiro.")
    if numero_decimal < 0:
        raise ValueError("A função não suporta números decimais negativos.")

    # Converter o número decimal para binário e remover o prefixo '0b'
    numero_binario = bin(numero_decimal)[2:]

    # Verifica se o número binário cabe no tamanho de bits desejado
    if len(numero_binario) > tamanho_bits:
        raise ValueError(f"O número binário '{numero_binario}' excede o tamanho de bits especificado ({tamanho_bits} bits).")

    # Adiciona zeros à esquerda para completar o tamanho de bits desejado
    numero_binario = numero_binario.zfill(tamanho_bits)

    # Inicializa o vetor com 1 para o produto de Kronecker
    vetor = np.array([1])
    
    # Aplica o produto de Kronecker para cada dígito binário
    for digito in numero_binario:
        vetor = np.kron(vetor, mapeamento[digito])

    return vetor

In [12]:
saida_tabela = [
	[1],
	[1],
	[0],
	[1],
]

vetores_binarios = []

for comb in saida_tabela:
	vetor = kronecker_binary(np.array([decimal_para_binario_vetor(bit, 1) for bit in comb]))
	vetores_binarios.append(vetor)

vetores_binarios = np.array(vetores_binarios).T
sp.Matrix(vetores_binarios)

Matrix([
[0, 0, 1, 0],
[1, 1, 0, 1]])

In [12]:
saida_tabela = [
    [0,0],
    [0,1],
	[0,1],
	[1,0]
]

saida_tabela = [linha[::-1] for linha in saida_tabela]

vetores_binarios = []

for comb in saida_tabela:
	vetor = kronecker_binary(np.array([decimal_para_binario_vetor(bit, 1) for bit in comb]))
	vetores_binarios.append(vetor)

vetores_binarios = np.array(vetores_binarios).T
sp.Matrix(vetores_binarios)

Matrix([
[1, 0, 0, 0],
[0, 0, 0, 1],
[0, 1, 1, 0],
[0, 0, 0, 0]])

In [13]:
regra = sp.Matrix(np.dot(vetores_binarios, variaveis)) # expressões simbólicas
regra

Matrix([
[                x2*x3],
[x1*x3 + x1*x4 + x2*x4]])

In [11]:
# Iterando pelo vetor com o índice e o valor
for linha, elemento in enumerate(regra):
    print(f"Linha {linha}: {elemento}")

Linha 0: x1*x3 + x1*x4 + x2*x3
Linha 1: x2*x4


In [12]:
lista = regra.tolist()[0]
list(set([x for x in lista if lista.count(x) > 1]))

[]

In [13]:
np.dot(vetores_binarios, decimal_para_binario_vetor(2, 4))

ValueError: shapes (2,4) and (16,) not aligned: 4 (dim 1) != 16 (dim 0)

In [ ]:
from sympy.utilities.lambdify import lambdastr

#  Lista para armazenar as funções
funcoes = []

# Variáveis que as funções irão receber
variaveis = criar_vetor_de_simbolos(bits*2)

# Iterar sobre cada elemento do vetor simbólico e criar uma função
for expr in regra:
    func = sp.lambdify(variaveis, expr)
    funcoes.append(func)


In [ ]:
from sympy.utilities.lambdify import lambdastr

# Variáveis que as funções irão receber
variaveis = criar_vetor_de_simbolos(bits*2)

for expr in regra:
    print(lambdastr(variaveis, expr))

lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (x1*x3*x5*x8)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (x1*x4*x6*x8)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x

In [ ]:
import itertools

# Usando produto cartesiano
combinacoes = list(itertools.product([0, 1], repeat=4))

for comb in combinacoes:
	# Converte cada bit da combinação para um vetor binário e achata em uma única dimensão
	valores = np.array([decimal_para_binario_vetor(bit, 1) for bit in comb]).flatten()
	print(valores,[f(*valores) for f in funcoes])

[1 0 1 0 1 0 1 0] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1 0 1 0 1 0 0 1] [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1 0 1 0 0 1 1 0] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0